In [1]:
import re
import pandas as pd

chat_file = "WhatsappChat.txt"

# WhatsApp message regex pattern
pattern = r"^(\d{1,2}/\d{1,2}/\d{2,4}), (\d{1,2}:\d{2}) - (.*?): (.*)$"

messages = []
with open(chat_file, "r", encoding="utf-8") as f:
    current_message = None
    for line in f:
        match = re.match(pattern, line)
        if match:
            date, time, sender, text = match.groups()
            messages.append({
                "date": date,
                "time": time,
                "sender": sender,
                "text": text.strip()
            })
        else:
            # Continuation of previous message
            if messages:
                messages[-1]["text"] += " " + line.strip()

df = pd.DataFrame(messages)

In [2]:
from agents import FileSearchTool, WebSearchTool, Agent, ModelSettings, TResponseInputItem, Runner, RunConfig
from pydantic import BaseModel

# Tool definitions
file_search = FileSearchTool(
  vector_store_ids=[
    "vs_68e665b3074c8191b6655dc14c46dca0"
  ]
)
web_search_preview = WebSearchTool(
  search_context_size="medium",
  user_location={
    "type": "approximate"
  }
)
class GreetingSchema(BaseModel):
  training_goal: str
  event_info: str


greeting = Agent(
  name="Greeting",
  instructions="""You are an AI assistant running coach. First you will greet the runner, and ask for any query.

Look through the conversation to extract the following:
1. Training goal(is there a question related to any aspect related to training)
2. Target Event (a query related to a running event)

If the above details are present anywhere in the conversation, return:
{
  \"training goal\": <user-provided goal>,
  \"event info\": \"<Running Event name>\",
 }""",
  model="gpt-4.1-mini",
  output_type=GreetingSchema,
  model_settings=ModelSettings(
    temperature=1,
    top_p=1,
    max_tokens=2048,
    store=True
  )
)


training_info = Agent(
  name="Training Info",
  instructions="Refer to the attached WhatsApp chat of a running group to answer questions",
  model="gpt-4.1-mini",
  tools=[
    file_search
  ],
  model_settings=ModelSettings(
    temperature=1,
    top_p=1,
    max_tokens=2048,
    store=True
  )
)


event_name = Agent(
  name="Event Name",
  instructions="Search for the requested information regarding the event from the internet and provide the answer",
  model="gpt-4.1-mini",
  tools=[
    web_search_preview
  ],
  model_settings=ModelSettings(
    temperature=1,
    top_p=1,
    max_tokens=2048,
    store=True
  )
)


agent = Agent(
  name="Agent",
  instructions="Courteously reply that you don't know the answer to the query",
  model="gpt-4o-mini",
  model_settings=ModelSettings(
    temperature=1,
    top_p=1,
    max_tokens=2048,
    store=True
  )
)


class WorkflowInput(BaseModel):
  input_as_text: str


# Main code entrypoint
async def run_workflow(workflow_input: WorkflowInput):
  state = {

  }
  workflow = workflow_input.model_dump()
  conversation_history: list[TResponseInputItem] = [
    {
      "role": "user",
      "content": [
        {
          "type": "input_text",
          "text": workflow["input_as_text"]
        }
      ]
    }
  ]
  greeting_result_temp = await Runner.run(
    greeting,
    input=[
      *conversation_history
    ],
    run_config=RunConfig(trace_metadata={
      "__trace_source__": "agent-builder",
      "workflow_id": "wf_68e4efaf64ac8190936826ade60fd3910d7efe675e39fbd9"
    })
  )

  conversation_history.extend([item.to_input_item() for item in greeting_result_temp.new_items])

  greeting_result = {
    "output_text": greeting_result_temp.final_output.json(),
    "output_parsed": greeting_result_temp.final_output.model_dump()
  }
  if greeting_result["output_parsed"]["training_goal"] != "":
    training_info_result_temp = await Runner.run(
      training_info,
      input=[
        *conversation_history
      ],
      run_config=RunConfig(trace_metadata={
        "__trace_source__": "agent-builder",
        "workflow_id": "wf_68e4efaf64ac8190936826ade60fd3910d7efe675e39fbd9"
      })
    )

    conversation_history.extend([item.to_input_item() for item in training_info_result_temp.new_items])

    training_info_result = {
      "output_text": training_info_result_temp.final_output_as(str)
    }
  elif greeting_result["output_parsed"]["training_goal"] != "":
    event_name_result_temp = await Runner.run(
      event_name,
      input=[
        *conversation_history
      ],
      run_config=RunConfig(trace_metadata={
        "__trace_source__": "agent-builder",
        "workflow_id": "wf_68e4efaf64ac8190936826ade60fd3910d7efe675e39fbd9"
      })
    )

    conversation_history.extend([item.to_input_item() for item in event_name_result_temp.new_items])

    event_name_result = {
      "output_text": event_name_result_temp.final_output_as(str)
    }
  else:
    agent_result_temp = await Runner.run(
      agent,
      input=[
        *conversation_history
      ],
      run_config=RunConfig(trace_metadata={
        "__trace_source__": "agent-builder",
        "workflow_id": "wf_68e4efaf64ac8190936826ade60fd3910d7efe675e39fbd9"
      })
    )

    conversation_history.extend([item.to_input_item() for item in agent_result_temp.new_items])

    agent_result = {
      "output_text": agent_result_temp.final_output_as(str)
    }


In [11]:
abc =await run_workflow(WorkflowInput(input_as_text="Which is the best diet for a marathon runner?"))

/var/folders/51/13r9fnld6y3fs0kwwl7nkjqh0000gn/T/ipykernel_1575/3101236305.py:125: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  "output_text": greeting_result_temp.final_output.json(),


In [12]:
print(abc)

None


In [3]:
# Make sure you import the necessary classes
from Whatsapp import run_workflow, WorkflowInput

# Run the workflow and capture the result
abc = await run_workflow(WorkflowInput(input_as_text="What is the best way to recover from injury?"))
print("abc contains:", abc)
print("Output text:", abc["output_text"] if abc else "No result")

abc contains: {'output_text': 'The best way to recover from an injury, especially a major running muscle or fascia injury (e.g., hamstring, ITB, calf, quad, ankle), involves a structured approach:\n\n1. **Stop and Assess Immediately**: Stop the activity to assess the injury. Apply the RICE principle (Rest, Ice, Compression, Elevation).\n   - Rest to avoid further injury.\n   - Ice the area to reduce blood flow and numb pain (ideally thrice a day for at least 10 minutes).\n   - Compression as needed but not while sleeping.\n   - Elevate the injured area to improve blood flow for healing.\n\n2. **Seek Medical Attention**: Consult a medical professional for proper diagnosis and rehabilitation plan. This is critical to differentiate between strains, tendonitis, ligament tears, or other conditions.\n\n3. **Modify and Continue Training**: Once cleared by a doctor or physio, work with your coach to modify training. This involves:\n   - Reducing impact on the injured part while maintaining ove